In [8]:
from typing import TypedDict,Annotated
from dotenv import load_dotenv
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,END,add_messages,START
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
)
search_tool = TavilySearchResults(max_results=4)
llm_with_tools = llm.bind_tools([search_tool])

In [9]:
class ChildState(TypedDict):
    messages: Annotated[list,add_messages]

In [10]:
def llm_node(state: ChildState):
    return {
        "messages":[llm_with_tools.invoke(state["messages"])]
    }
def conditional_edge(state: ChildState):
 return "tool_node" if hasattr(state["messages"][-1],"tool_calls") and len(state["messages"][-1].tool_calls)>0 else END

In [11]:
tool_node = ToolNode([search_tool],messages_key="messages")
child_graph = StateGraph(ChildState)
child_graph.add_node("llm_node",llm_node)
child_graph.add_node("tool_node",tool_node)
child_graph.add_conditional_edges("llm_node",conditional_edge,{"tool_node":"tool_node",END:END})
child_graph.add_edge("tool_node","llm_node")
child_graph.set_entry_point("llm_node")
child_app = child_graph.compile()

In [12]:
class ParentState(TypedDict):
    messages: Annotated[list,add_messages]

In [15]:
parent_graph = StateGraph(ParentState)
parent_graph.add_node("child_agent",child_app)

parent_graph.add_edge(START,"child_agent")
parent_graph.add_edge("child_agent",END)

parent_app = parent_graph.compile()

parent_app.invoke({"messages":["whats the weather like in vallabhapuram today, and weather in banglore yesterday"]})

{'messages': [HumanMessage(content='whats the weather like in vallabhapuram today, and weather in banglore yesterday', additional_kwargs={}, response_metadata={}, id='929fd0c7-9642-49f9-8fb3-70ecfade08f6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wqyqywn5b', 'function': {'arguments': '{"query":"current weather in Vallabhapuram today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': '3ggrbcg6k', 'function': {'arguments': '{"query":"Banglore weather yesterday"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 595, 'total_tokens': 641, 'completion_time': 0.061333333, 'prompt_time': 0.035325389, 'queue_time': -0.09202105899999999, 'total_time': 0.096658722}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--832401eb-5324-4d55-8b66-3edbd6b0abc0-0', tool_calls=[{'name': 

### When parent and child have different schemas

In [33]:
class ParentState2(TypedDict):
    query: str
    response : str


In [34]:
def intermediate_node(state:ParentState2):
    response = child_app.invoke(
        {
            "messages":state["query"]
        }
    )
    state["response"] = response["messages"][-1].content
    return state


In [37]:
parent_graph_2 = StateGraph(ParentState2)
parent_graph_2.add_node("child_agent",intermediate_node)

parent_graph_2.add_edge(START,"child_agent")
parent_graph_2.add_edge("child_agent",END)

parent_graph_2 = parent_graph_2.compile()

parent_graph_2.invoke({"query":"whats the weather like in vallabhapuram today, and weather in banglore yesterday","response":""})

{'query': 'whats the weather like in vallabhapuram today, and weather in banglore yesterday',
 'response': 'The weather in Vallabhapuram today is mostly cloudy with occasional showers and a high of 98F. \n\nThe weather in Bangalore yesterday was pleasant with temperatures ranging from 70F to 86F and scattered clouds.'}